**Умова:**

Випадкова величина $X$ розподілена рівномірно. Знайти щільність її розподілу, якщо $P(X \geqslant 3)0,4$ а математичне сподівання рівне $M(X) = 2$

**Розв'язок:**

Щільність розподілу визначається за формулою:

$$f(x) =\cases{0, x \leq a \\\frac{1}{b−a},  a < x\leq b\\0, x > b}  ,звідки $$

                                       

$$P(c < X < d) = \int_c^d\frac{1}{b - a} ∙ {dx}= \frac{d-c}{b-a} = 0,4$$

Оскільки $c = 3$, а $d = b$, маємо:

$$\frac{b-3}{b-a} = 0,4$$

Математичне сподівання  визначається за формулою:
    
$$M(X) = \frac{a + b}{2} = 2$$    

Складемо систему рівнянь:

$$\cases{\frac{b-3}{b-a} = 0,4;  \\ \frac{a+b}{2} = 2;}$$


$$\cases{{b-3} = 0,4b - 0,4a;  \\ a+b = 4;}$$

$$\cases{0,6b + 0,4a = 3;  \\ a = 4 - b;}$$

$$\cases{a = -3;  \\ b = 7;}$$

Звідки щільність розподілу даної рівномірної величини має вигляд:

$$f(x) = \cases{0, x \leq - 3;  \\ 0,1, - 3 < x \leq 7; \\ 0, x > 7}$$

що означає, що випдкова величина $X$ у даному інтервалі від -3 до 7 має значення 0,1. За межами даного інтервалу $X = 0$